## Leitura e processamento dos dados no formato hdf

Versão: 0.1 



In [ ]:
# ---------------- LEGACY ---------------------------------
# code 1: opening as a netCDF4
# (example from https://hdfeos.org/zoo/index_openGESDISC_Examples.php)
# ---------------------------------------------------------

# import os

# # import matplotlib as mpl
# # import matplotlib.pyplot as plt
# # from mpl_toolkits.basemap import Basemap
# import numpy as np
# from netCDF4 import Dataset
# my_path = '/home/mapbiomasar/Mayra/AIRS/AIRS_L3/CH4_CO_L3_2002_today/'  
# my_filename = 'AIRS.2022.02.16.L3.RetStd_IR001.v6.0.32.0.G22048195935.hdf'  
# nc = Dataset(my_path + my_filename)



In [4]:

# initialization cell
# reads the selected columns in hdf files using the python wrapper (pyhdf)

import numpy as np
import pandas as pd

from pyhdf.SD import SD, SDC


my_path = '/home/mapbiomasar/Mayra/AIRS/AIRS_L3/CH4_CO_L3_2002_today/'


my_filename = 'AIRS.2012.08.30.L3.RetStd_IR001.v6.0.9.0.G14056074243.hdf'
hdf = SD(my_path+my_filename, SDC.READ)

# # Uncomment to list available SDS datasets
# print (hdf.datasets())

# # # # # # #  READ VARIABLE FROM HDF FILE  # # # # # # #  
def GetHDFData(data1,data2):
    
    # extract variables of interest
    data3D = hdf.select(data1)
    data1 = data3D[0,:,:]

    data3D = hdf.select(data2)
    data2 = data3D[0,:,:]
    
    # Read geolocation dataset.
    lat = hdf.select('Latitude')
    latitude = lat[:,:]
    lon = hdf.select('Longitude')
    longitude = lon[:,:]

    # reshape, and transpose values
    data = np.array([latitude.reshape(-1),
                    longitude.reshape(-1),
                    data1.reshape(-1),
                    data2.reshape(-1)]).transpose()

    # ----- Uncomment this code to change the -9999 to N/A    
    
    # Handle fill value.
    attrs = data3D.attributes(full=1)
    fillvalue=attrs["_FillValue"]

    # fillvalue[0] is the attribute value.
    fv = fillvalue[0]
    data[data == fv] = np.nan
    data = np.ma.masked_array(data, np.isnan(data))
    
    # ----- end handle N/A s
 
    return data
# # # # # # #  END OF READER  # # # # # # #  

# # get data and build dataframe 
# data1 = 'CO_VMR_A'
# data2 = 'CO_VMR_D'
# co_meas = GetHDFData(data1,data2)
# df  = pd.DataFrame(co_meas, columns = ['lat','lon','co_asc','co_desc'])

# # stacking the monthly values
# # TODO

# # print(co_desc.shape)
# # temp  = np.vstack([co_desc,co_asc])
# # print(temp.shape)

# co_meas.shape



In [5]:
# this loop will read all the files in a per month basis and aggregate
import glob
import time
yr = list(range(2015,2021))
mon = ['01','02','03','04','05','06',
       '07','08','09','10','11','12']
# mon = ['07','08', '09']
# yr = ['2003','2013']
data1 = 'CO_VMR_A'
data2 = 'CO_VMR_D'


start_time = time.time()
# df = pd.DataFrame(columns = ['lat','lon','co_asc','co_desc'])
for y in yr:
    print('PROCESSING: ', y, ' files')
    for m in mon:
        path_file = my_path + 'AIRS.' + str(y) + '.'+ m +'.' + '*.hdf'
        
        # gambi 1: zera o vetor que armazena os valores mensais
        # define o novo df que sera exportado
        tmp = np.empty([1,4])
        tmp[:] = np.nan
        df = pd.DataFrame(columns = ['lat','lon','co_asc','co_desc'])
        
        for filename in glob.glob(path_file):
            # print(filename)
            try:
                hdf = SD(filename, SDC.READ)
                co_meas = GetHDFData(data1,data2)
                tmp  = np.vstack([tmp,co_meas])
            except:
                print("Could not open ", filename)
                break
        # build df with the time information
        df2 = pd.DataFrame(tmp,columns = ['lat','lon','co_asc','co_desc'])
        df2['time'] = str(y) + '-' + m
        df = pd.concat([df,df2])
        df = df.dropna()
            
        # process the monthly files and export to CSV
        df_co = df.groupby(['time','lat','lon']).mean().reset_index()
        df_co.to_csv(my_path + 'CO_' + str(y) + '_' + m + '.csv')
        df.co = None
print('Processing time:', (time.time()-start_time)/60 , ' min')

# df2['time'] = '2022-06'
# df2


PROCESSING:  2015  files
PROCESSING:  2016  files


HDF4Error: SD (7): Error opening file

### TODO: verificar o problema que deu no arquivo de 2014



In [23]:
print(df.shape)
print(df2.shape)

(2199498, 5)
(185231, 5)


In [24]:
185231/2199498

0.08421512545135298

In [ ]:
# build geopandas dataframes

import geopandas as gpd 
from shapely.geometry import Polygon, Point, MultiPolygon

crs = {'init':'EPSG:4326'}
geometry = [Point(xy) for xy in zip(df_asc['lon'], df_asc['lat'])]

points = gpd.GeoDataFrame(df_asc, 
                          crs = crs, 
                          geometry = geometry)

# optional: filter to a square covering South America

min_lat = -57.5858
max_lat =  15.5988
min_lon = -85.8360
max_lon = -35.8750

# back to original nomenclature
tmp = points.copy()

lat_filter = (tmp["lat"] >= min_lat) & (tmp["lat"] <= max_lat)
lon_filter = (tmp["lon"] >= min_lon) & (tmp["lon"] <= max_lon)

# tmp = tmp.loc[lat_filter & lon_filter]

points = tmp[lat_filter & lon_filter]


In [ ]:
buffers = points.buffer(0.5) # 1 x 1 degree grid
bounds = buffers.bounds

bounds['pixel_area'] = bounds.apply(
  lambda obj: Polygon(shell=[
    Point(obj['maxx'],obj['miny']),
    Point(obj['minx'],obj['miny']),
    Point(obj['minx'],obj['maxy']),
    Point(obj['maxx'],obj['maxy']),
    Point(obj['maxx'],obj['miny']),
  ]),
  axis=1
)

# adding indexes to merge dataframes
points.insert(0, 'New_ID', range(1, 1 + len(points)))
bounds.insert(0, 'New_ID', range(1, 1 + len(bounds)))

my_merge = bounds.merge(points, on='New_ID')

In [ ]:
my_merge

In [ ]:
def ToShapeFile(gdf,yr,mon):
  gdf = gpd.GeoDataFrame(
    gdf,
    crs = {'init':'EPSG:4326'}, 
    geometry = [a for a in gdf['pixel_area']]                               
    )
  gdf = gdf.reset_index()
  gdf = gdf.loc[:,gdf.columns.isin(['xch4', 'geometry'])]
  gdf.to_file('/home/mapbiomasar/MJT/shapefiles/xch4_merged_mips' + str(yr) +'_' + str(mon) + '.shp')

Aqui começa o processamento do lote. Primeiramente devo agregar os valores por mes para finalmente montar os shapefiles

In [ ]:
import datetime
import time
# do not show these annoying warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


df = my_merge.copy()

df['Date'] = pd.to_datetime(df['time'])
df = df.set_index(df['Date'])
df = df.sort_index()
df = df[['lat','lon','pixel_area','xch4']]

year = list(range(2003,2021))
month = list(range(1,13))
start_time = time.time()
for yr in year:
    print('Processing ', yr, ' shapefiles')
    for mon in month:
        # shapefile_name = '/home/mapbiomasar/MJT/shapefiles/oco2_merged_mips' + str(yr) +'_' + str(mon) +'.shp'
        # print(shapefile_name)
        # we keep within a month
        date_min = datetime.datetime(yr, mon, 1)
        date_max = datetime.datetime(yr, mon, 25)
        to_process = df[date_min:date_max]
        if to_process.empty:        
            print('Dataframe from ', str(yr)+str(mon), 'is empty!') 
        else:
            ToShapeFile(to_process,yr,mon)

print('Processing time:', time.time()-start_time)

In [ ]:
df

In [ ]:
my_merge


In [ ]:
# # new geopandas with the variables of interest
# gdf = my_merge.copy()
# gdf = gpd.GeoDataFrame(
#   gdf,
#   crs = crs, 
#   geometry = [a for a in gdf['pixel_area']]                               
#   )
# gdf = gdf.reset_index()
# gdf = gdf.loc[:,gdf.columns.isin(['xco2', 'geometry'])]

# # print('--> TYPE: ',type(gdf))
# gdf


In [ ]:
# testing the view
import matplotlib.pyplot as plt

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(figsize=(18, 7))


# ax.set_xlim(178.0, 181.0)
# ax.set_ylim(20.0, 23.0)
world.plot(ax=ax, alpha=0.8, color='grey')

gpd.GeoDataFrame(
  my_merge,
  crs = crs, 
  geometry = [a for a in my_merge['pixel_area']]                                
  ).plot(column='co_asc', ax=ax, legend=True,alpha = 0.4)

# gpd.GeoDataFrame(
#   points,
#   crs = crs, 
#   geometry = [a for a in points['geometry']]                                
#   ).plot(column='xco2', ax=ax)



# plt.title('xco2 from Mayra')


In [ ]:


  print('--> TYPE: ',type(gdf))



### Part 2: reprocessing and splitting files

TODO: separar os arquivos por meses para subir ao GEE



In [ ]:
# # summarizing table
# print(len(my_merge))
# tmp = my_merge.groupby(['time'])['xco2'].mean()
# len(tmp)

In [ ]:
# looping per year and month


In [ ]:
my_merge

In [ ]:
print('--> ',type(gdf))

gdf.to_file('/home/mapbiomasar/MJT/notebooks/smallGridMerged.shp')
